In [ ]:
import math
import os
import sys

from tqdm import tqdm

import torch
import torchvision.transforms as transforms
import torch.utils.data as torch_data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
from shapely.geometry.point import Point
from skimage.draw import circle_perimeter_aa
import matplotlib.pyplot as plt

import cv_practical.main as cvp_utils

##### Create Dataset

In [ ]:
class CIRCLEDataset(torch_data.Dataset):
    
    def __init__(self, count=1000, debug = False, transform=None):
        self._deubg = debug
        self._count = count
        self._circle_images, self._circle_params = [], []
        
        self.transform = transforms
        
        self.create_dataset()
        
    def create_dataset(self):
        for i in range(self._count):
            # size, radius, noise
            params, img = noisy_circle(200, 200, 2)
            self._circle_images.append(
                np.expand_dims(img, axis=0)
            )
            self._circle_params.append(
                np.asarray(params, dtype = np.float32)
            )
            
    def __len__(self):
        return len(self._circle_images)
    
    def __getitem__(self, idx):
        return [self._circle_images[idx], self._circle_params[idx]]
        
     

##### Create network

In [ ]:
class CDNet(nn.Module):
    def __init__(self, in_planes):
        super(CDNet, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, 6, kernel_size = 2, stride = 2, bias = False)
        self.conv2 = nn.Conv2d(6, 16, kernel_size = 5, stride = 1)
        self.fc1   = nn.Linear(16*23*23, 4*23*23)
        self.fc2   = nn.Linear(4*23*23, 23*23)
        self.fc3   = nn.Linear(23*23, 3)

    def forward(self, x):
        b_dx, c_dx, w, h = x.size()
        out = self.conv1(x)
        out = F.max_pool2d(out, 2)
        
        out = self.conv2(out)
        out = F.max_pool2d(out, 2)
        
        out = out.view(b_dx, -1)
        
        out = self.fc1(out)
        out = F.relu(out)
        
        out = self.fc2(out)
        out = F.relu(out)
        
        out = self.fc3(out)
        
        return out

In [ ]:
def train(model, optimizer, criterion, device, dataloader):
    model.train()
    train_loss = 0.0
    tbar = tqdm(dataloader)
    num_samples = len(dataloader)
    for i, sample in enumerate(tbar):
        image, target = sample[0].float(), sample[1].float()
        image, target = image.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        tbar.set_description('Train loss:  %.3f' % (train_loss / (i + 1)))
    return train_loss

def validate(model, criterion, device, dataloader):
    model.eval()
    val_loss = 0.0
    tbar = tqdm(dataloader)
    num_samples = len(dataloader)
    with torch.no_grad():
        for i, sample in enumerate(tbar):
            image, target = sample[0].float(), sample[1].float()
            image, target = image.to(device), target.to(device)

            output = model(image)
            loss = criterion(output, target)
            val_loss += loss.item()
            tbar.set_description('Val loss:    %.3f' % (train_loss / (i + 1)))
    return val_loss


def test(model, device, dataloader):
    model.eval()
    val_loss = 0.0
    tbar = tqdm(dataloader)
    num_samples = len(dataloader)
    outputs = []
    with torch.no_grad():
        for i, sample in enumerate(tbar):

            image = sample[0].float()
            image = image.to(device)
            outputs.append([sample[0], model(image), sample[1]])
            tbar.set_description('Test progress:%.2f' % (train_loss / (i + 1)))

    return outputs


In [ ]:
if __name__=="__main__":
    """
    print("torch.cuda.is_available()   =", torch.cuda.is_available())
    print("torch.cuda.device_count()   =", torch.cuda.device_count())
    print("torch.cuda.device('cuda')   =", torch.cuda.device('cuda'))
    print("torch.cuda.current_device() =", torch.cuda.current_device())
    print()
    """
    epochs = 400
    device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

    train_dataset = CIRCLEDataset(
        count = 10000,
        debug=False,
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
    )
    val_dataset = CIRCLEDataset(
        count = 1000,
        debug=False,
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
    )
    
    test_dataset = CIRCLEDataset(
        count = 1000,
        debug=False,
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
    )


    train_dataloader = torch_data.DataLoader(train_dataset, num_workers=0, batch_size=32)
    val_dataloader = torch_data.DataLoader(val_dataset, num_workers=0, batch_size=32)
    test_dataloader = torch_data.DataLoader(test_dataset, num_workers=0, batch_size=32)

    model = CDNet(in_planes = 1)
    if torch.cuda.device_count() > 1:
        print("Using ", torch.cuda.device_count(), " GPUs!")
        model = nn.DataParallel(model)

    model.to(device)

    optimizer = torch.optim.SGD(
        lr=0.001,
        momentum=0.9,
        weight_decay=5e-4,
        nesterov=False,
        params=model.parameters()
    )

    criterion = nn.MSELoss()

    for epoch in range(epochs):
        train_loss = train(model, optimizer, criterion, device, train_dataloader)
        val_loss = validate(model, criterion, device, val_dataloader)
        outputs = test(model, device, test_dataloader)

        counter = 0
        
        state = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        """
        model_save_str = './results/models/{}-{}-{}.{}'.format(
            "segnet", "bn2d", epoch, "pth"
        )"""

        torch.save(
            state,
            model_save_str
        )
        print(epoch, train_loss, val_loss)